# 03 Parameter estimation

Spectral kernels are flexible but difficult to optimize, to solve this we equip the toolkit with different ways to initialize the parameters before training.

In this notebook we will showcase the diferent initialization for single output spectral mixture and multioutput kernels.

For single output spectral mixture kernel there is 3 ways:
* __Random__: Uses heuristic defined in ndrew Wilson PHD thesis, 
    - Inverse of lengthscales should be drawn from truncated Gaussian |N(0, max_dist^2)|
    - Draw means from Unif(0, 0.5 / minimum distance between two points)
    - Mixture weights should be roughly the std  of the y values divided by the number of mixtures
    - 
* __BNSE__: Uses Bayesian Nonparametric Spectral Estimation (Tobar 2018) to estimate the power spectral density (PSD) of the signal, and asign.
    - Find the peaks in the PSD and order it by magnitude
    - Means as the position of the first Q peaks
    - Lengthscales as the width of the peaks.
    - Mixture weight as the normalized peaks magnitude.
    

* __Lomb Scargle__: Uses Lomb Scargle periodogram and obtain an estimate of the PSD, then follow the same heuristic as BNSE.


For multiputput spectral kernels there is 2 ways:

* __SM__: Fit and independent GP with spectral mixture kernel for each channel, then use those parameters as initial parameters.

* __BNSE__: For each channel estimate the PSD using BNSE, and apply the same heuristics as the single output case.

In both cases the noise for each channel is initializated as $1/30$ of the channels variance.

In [1]:
# import library if it is not installed
import sys
sys.path.insert(0, '../')

import mogptk
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%reload_ext autoreload
%autoreload 2

In [2]:
sns.set_context('notebook', font_scale=1.3)
sns.set_style('ticks')
plt.rcParams['figure.figsize'] = (10, 4)

## Synthetic  Data

To showcase the initialization, we first create a dataset consisting in a sum of sinuoids

$y(t) = \sum_{i=1}^{3} a_i \sin(t f_i) + \epsilon, \quad \epsilon \sim \mathcal{N}(0, \sigma_n^2)$

With known frequencies $f_i$ and amplitudes $a_i$, when fiting the spectral mixture kernel, we should should expect the means to have similar values as the frequencies

In [ ]:
# target freq to estimate
target_freqs = np.array([0.2, 1, 2])
target_amps = [1, 0.5, 0.5]

n_points = 500

t = np.linspace(0, 20, n_points)
y = np.zeros(n_points)

for i in range(3):
    y += target_amps[i] * np.sin(2 * np.pi * target_freqs[i] * t)
    
y_n = y + np.random.normal(scale=0.4, size=n_points)
y_n = np.exp(y_n)
y_n += -y_n.min() + 1e-1

# data class
data = mogptk.Data(t.reshape(-1, 1), y_n)
data.remove_range(start=10.0, end=None)

data.plot()

In [ ]:
data.plot_spectrum()

In [ ]:
data.transform(mogptk.TransformLog)
data.plot()

In [ ]:
# create model
model = mogptk.SM([data], Q=4)

model.plot_psd(title='PSD with random parameters')

In [ ]:
# initialize params
method = 'BNSE'
model.init_params(method=method)

model.plot_psd(title='PSD with {} initialization'.format(method))

In [ ]:
model.build()
model.train(method='L-BFGS-B', maxiter=5000, tol=1e-20)

model.plot_psd(title='PSD with model trained')

In [ ]:
y_pred, lower_ci, upper_ci  = model.predict([t])

plt.plot(t, y_pred[0], label='Post.mean', lw=2, zorder=2)
plt.plot(t[:250], y_n[:250], '.k', alpha=0.7, zorder=2, label='Train')
plt.plot(t[250:], np.exp(y[250:]), '--k', zorder=3, label='Test')
plt.fill_between(t,
                 lower_ci[0],
                 upper_ci[0],
                 color='b',
                 alpha=0.3,
                 zorder=1,
                 label='95% c.i')
plt.xlim(-0.1, 20.1)
plt.legend(ncol=4)
plt.title('Predictions with trained model');

In [ ]:
# err_param, err_pred = experiment(t[:250], y_n[:250], t, y, target_freqs, Q=3, n_trials=10)

In [ ]:
# err_param.mean(1), err_param.std(1)

In [ ]:
# err_pred.mean(1), err_pred.std(1)


# Experiment 1: Mauna Loa

In [ ]:
# dataset mauna loa
from sklearn.datasets import fetch_openml

def load_mauna_loa_atmospheric_co2():
    ml_data = fetch_openml(data_id=41187)
    months = []
    ppmv_sums = []
    counts = []

    y = ml_data.data[:, 0]
    m = ml_data.data[:, 1]
    month_float = y + (m - 1) / 12
    ppmvs = ml_data.target

    for month, ppmv in zip(month_float, ppmvs):
        if not months or month != months[-1]:
            months.append(month)
            ppmv_sums.append(ppmv)
            counts.append(1)
        else:
            # aggregate monthly sum to produce average
            ppmv_sums[-1] += ppmv
            counts[-1] += 1

    months = np.asarray(months).reshape(-1)
    avg_ppmvs = np.asarray(ppmv_sums) / counts
    return months, avg_ppmvs

In [ ]:
x, y = load_mauna_loa_atmospheric_co2()

stop = 200

plt.plot(x[:stop], y[:stop], label='Train')
plt.plot(x[stop:], y[stop:], 'r', label='Test')
plt.legend()
plt.xlabel('Year')
plt.ylabel('CO2');
plt.title('Mauna Loa dataset');

In [ ]:
# add data
data = mogptk.Data(x[:stop, np.newaxis], y[:stop])
data.transform(mogptk.TransformDetrend(3))

# create model
model = mogptk.SM([data], Q=10)

model.plot_psd(title='PSD with random parameters')

In [ ]:
# plot detrended data
data.plot(title='Detrended Data')

In [ ]:
method = 'BNSE'
model.init_params(method)
model.plot_psd(title='PSD with {} initialization'.format(method))

In [ ]:
%%time 
model.build()
model.train(method='L-BFGS-B', maxiter=10000, tol=1e-85)

model.plot_psd(title='PSD with model trained')

In [ ]:
%%time 
# prediction
x_pred = [x.reshape(-1, 1)]

y_pred, lower_ci, upper_ci = model.predict(x_pred)

# plot
plt.plot(x[:stop], y[:stop], '-k', label='Train')
plt.plot(x[stop:], y[stop:], '-r', label='Test')
plt.plot(x_pred[0], y_pred[0], lw=1.5, zorder=1, label='GP-SM')
plt.fill_between(x_pred[0].reshape(-1),
                 lower_ci[0],
                 upper_ci[0],
                 color='b',
                 alpha=0.3,
                 zorder=1)
plt.legend(ncol=3)
plt.title('Mauna Loa')
plt.xlabel('Year')
plt.ylabel('CO2')

# Airplane passangers

In [ ]:
air = np.loadtxt('data/Airline_passenger.csv')

stop = 96

x = air[: ,0]
y = air[:, 1]

# add data
data = mogptk.Data(x[:stop, np.newaxis], y[:stop])

data.transform(mogptk.TransformLog)
data.transform(mogptk.TransformDetrend(3))

# create model
model = mogptk.SM([data], Q=10)

data.plot()

In [ ]:
method = 'BNSE'
model.init_params(method)

model.plot_psd(title='PSD with {} initialization'.format(method))

In [ ]:
model.build()
model.train('L-BFGS-B', maxiter=2000, tol=1e-80)

model.plot_psd(title='PSD with model trained')

In [ ]:
x_pred = [x[:, np.newaxis]]
y_pred, lower_ci, upper_ci = model.predict(x_pred)

# plot
plt.plot(x[:stop], y[:stop], '-k', label='Train')
plt.plot(x[stop:], y[stop:], '-r', label='Test')
plt.plot(x_pred[0], y_pred[0], lw=1.5, zorder=2, label='GP-SM')
plt.fill_between(x_pred[0].reshape(-1),
                 lower_ci[0],
                 upper_ci[0],
                 color='b',
                 alpha=0.3,
                 zorder=1)
plt.title('Airline passanger')
plt.legend(ncol=3)
plt.xlabel('Month')
plt.ylabel('Passanger');

# Jura single output


Save for later (until multi input)